# 기본 정보 입력

In [1]:
# 패키지 불러오기
import openai

In [2]:
# API 키 지정하여 클라이언트 선언하기
client = openai.OpenAI(api_key = "발금받은 OpenAI API 키")

## 파일 업로드 및 벡터 저장소 추가하기

In [3]:
# 벡터 저장소 생성하기
vector_store = client.beta.vector_stores.create(name="축구 규칙 파일")

In [4]:
# 파일을 벡터 저장소에 올리기
file_streams = open("축구규칙정리.pdf", "rb")

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=[file_streams]
)
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


# Create Assistant
- instructions: 어시스턴트와 모델이 어떻게 행동하거나 응답해야 하는지 알려줍니다.
- model: 미세 조정된 모델을 포함하여 모든 GPT-3.5 또는 GPT-4 모델을 지정할 수 있습니다. 검색 도구에는 gpt-3.5-turbo-1106 및 gpt-4-1106-preview 모델이 필요합니다.
- tools: API는 OpenAI에서 빌드 및 호스팅하는 코드 인터프리터와 웹 검색을 지원합니다. 또한 함수 호출 기능과 유사한 동작으로 사용자 정의 함수 서명을 정의할 수 있습니다


In [7]:
instruction = '''
[목적]
이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.

[규칙]
1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.
2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.
3. 답변의 형태는 아래 예시와 같이 해주세요
예시)
-  질문 : 질문 내용
-  답변 : 답변내용 
4. 모든 질문에 한국어로 답변해주세요.
'''

In [8]:
assistant = client.beta.assistants.create(
  name = "축구 규칙 설명 챗봇",
  instructions=instruction,
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

print(assistant.id)

asst_ARpw27ITT2HMg78CnxwNLv7A


## 생성한 Assistant 업데이트 하기

In [ ]:
# Run this only if you need to update the configuration of the assistant
assistant = client.beta.assistants.update(
  assistant.id,
  instructions=instruction,
  model="gpt-4-1106-preview",
  #model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

## 기존 Assistant 불러오기

In [9]:
# 나의 Assistant 목록 확인
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_2zHoFZNdrDxlqksAmPTRAweO', created_at=1709470447, description=None, file_ids=['file-5u9FN54oWlCC1FpPi26IMAMY'], instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n예시)\n-  질문 : 질문 내용\n-  답변 : 답변내용 \n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-3.5-turbo-1106', name='축구 규칙 설명 챗봇', object='assistant', tools=[ToolRetrieval(type='retrieval')])]


In [4]:
assistant = client.beta.assistants.retrieve("asst_KoH4sLsw9B4vwjsKVbpmKy1l")
print(assistant)

Assistant(id='asst_KoH4sLsw9B4vwjsKVbpmKy1l', created_at=1702534045, description=None, file_ids=['file-wRakvw7TiO1b2rNUrh7kSsGc'], instructions="\nYou're a soccer expert explaining the rules of the game\n\nRules:\n1. when a user asks you a question about the rules of soccer, you find it in the uploaded file and answer it in detail.\n2. if it's not in the file, say you're not sure.\n3. answer all questions in Korean.\n", metadata={}, model='gpt-3.5-turbo-1106', name='축구 규칙 설명봇t', object='assistant', tools=[ToolRetrieval(type='retrieval')])


# Thread

## Thread 생성하기

In [9]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_ali5tRuErmPTC0MCdwlid3gh', created_at=1714955827, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


## Thread 에 메세지 넣기

In [10]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="축구장의 크기는?"
)
print(message)

Message(id='msg_qRtdjD1XVNjTwVqnsxEuputE', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='축구장의 크기는?'), type='text')], created_at=1714955831, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ali5tRuErmPTC0MCdwlid3gh')


# Assistant Run 하기

In [11]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [12]:
import time

while run.status not in ["completed", "failed"]:
  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id
  )
  print(run.status)
  time.sleep(5)

completed


# 결과확인(Run complete 후)

In [13]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [14]:
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: 축구장의 크기는 다음과 같습니다:
- 터치라인 길이: 최소 90m (100 야드) ~ 최대 120m (130 야드)
- 골라인 길이: 최소 45m (50 야드) ~ 최대 90m (100 야드)
- 국제경기를 위한 크기
  - 터치라인 길이: 최소 100m (110 야드) ~ 최대 110m (120 야드)
  - 골라인 길이: 최소 64m (70 야드) ~ 최대 75m (80 야드)

또한, 골대의 크기는 골대 내측의 거리가 7.32m(8야드)이고, 지면에서 크로스바 아래쪽까지의 높이는 2.44m(8피트)입니다. 이러한 기준은 FIFA의 규정에 따라서 정해지며, 대회 주최 측은 골라인 및 터치라인의 길이를 결정할 수 있습니다.

이 정보는 '축구규칙정리.pdf' 파일에서 찾았습니다. 
user: 축구장의 크기는?


# 질문 추가하기

In [15]:
# 메세지 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="오프사이드 룰에 대해 설명해줘"
)

# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

user: 오프사이드 룰에 대해 설명해줘
assistant: 축구장의 크기는 다음과 같습니다:
- 터치라인 길이: 최소 90m (100 야드) ~ 최대 120m (130 야드)
- 골라인 길이: 최소 45m (50 야드) ~ 최대 90m (100 야드)
- 국제경기를 위한 크기
  - 터치라인 길이: 최소 100m (110 야드) ~ 최대 110m (120 야드)
  - 골라인 길이: 최소 64m (70 야드) ~ 최대 75m (80 야드)

또한, 골대의 크기는 골대 내측의 거리가 7.32m(8야드)이고, 지면에서 크로스바 아래쪽까지의 높이는 2.44m(8피트)입니다. 이러한 기준은 FIFA의 규정에 따라서 정해지며, 대회 주최 측은 골라인 및 터치라인의 길이를 결정할 수 있습니다.

이 정보는 '축구규칙정리.pdf' 파일에서 찾았습니다. 
user: 축구장의 크기는?


In [16]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,)

while run.status not in ["completed", "failed"]:

  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id)

  print(run.status)

  time.sleep(5)

in_progress
in_progress
in_progress
completed


In [17]:
# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: 오프사이드는 축구에서 자주 논쟁거리가 되는 규칙 중 하나입니다. 오프사이드 규칙은 다음과 같습니다.

- 공격수가 볼을 받을 때 수비수보다 더 가까이에 있으면 오프사이드입니다.
- 그러나, 공격수가 골키퍼와 같거나 그보다 더 가까이에 있으면 오프사이드가 아닙니다.
- 또한, 볼을 받을 때 이미 오프사이드 위치에 있는 선수가 상대 선수를 방해하지 않는다면 오프사이드라고 할 수 없습니다.

이 정보는 '축구규칙정리.pdf' 파일에서 찾았습니다.    【4:4†source】 
user: 오프사이드 룰에 대해 설명해줘
assistant: 축구장의 크기는 다음과 같습니다:
- 터치라인 길이: 최소 90m (100 야드) ~ 최대 120m (130 야드)
- 골라인 길이: 최소 45m (50 야드) ~ 최대 90m (100 야드)
- 국제경기를 위한 크기
  - 터치라인 길이: 최소 100m (110 야드) ~ 최대 110m (120 야드)
  - 골라인 길이: 최소 64m (70 야드) ~ 최대 75m (80 야드)

또한, 골대의 크기는 골대 내측의 거리가 7.32m(8야드)이고, 지면에서 크로스바 아래쪽까지의 높이는 2.44m(8피트)입니다. 이러한 기준은 FIFA의 규정에 따라서 정해지며, 대회 주최 측은 골라인 및 터치라인의 길이를 결정할 수 있습니다.

이 정보는 '축구규칙정리.pdf' 파일에서 찾았습니다. 
user: 축구장의 크기는?


# Assistants 삭제

In [25]:
response = client.beta.assistants.delete(assistant.id)
print(response)

AssistantDeleted(id='asst_ARpw27ITT2HMg78CnxwNLv7A', deleted=True, object='assistant.deleted')


# File 삭제

In [26]:
# 업로드한 파일 목록 가져오기
file_list = client.files.list()
print(file_list)

SyncPage[FileObject](data=[FileObject(id='file-XHR2zl9rcLf20XEOiFy6NCEI', bytes=30066107, created_at=1714955622, filename='축구규칙정리.pdf', object='file', purpose='assistants', status='processed', status_details=None)], object='list', has_more=False)


In [27]:
# 특정 파일 ID 가져오기
file_id = file_list.data[0].id
print(file_id)

file-XHR2zl9rcLf20XEOiFy6NCEI


In [28]:
# 업로드 파일 삭제
response = client.files.delete(file_id)
print(response)

FileDeleted(id='file-XHR2zl9rcLf20XEOiFy6NCEI', deleted=True, object='file')
